In [ ]:
import numpy as np 
import pandas as pd 

def converterTempo(data):
    if(len(data) == 10):
        minutes = float(data[0:2])
        seconds = float(data[3:9])
        minutes = minutes*60
        
    if(len(data) == 9):
        minutes = float(data[0:1])
        seconds = float(data[2:8])
        minutes = minutes*60
    
    if(len(data) == 8):
        minutes = float(data[0:1])
        seconds = float(data[2:7])
        minutes = minutes*60
    
    return minutes + seconds

malhas = [1000, 2000, 3000, 4000, 5000]

temposFlagFast = []
temposFlagHost = []
temposFlagDevice = []
melhoresTemposPorMalha = []

eficienciasFlagFast = []
eficienciasFlagHost = []
eficienciasFlagDevice = []
melhoresEficienciasPorMalha = []

speedupsFlagFast = []
speedupsFlagHost = []
speedupsFlagDevice = []
melhoresSpeedupsPorMalha = []

# tab = '\t'

# dic = {"cores": malhas,
#        "tempo": temposNoFlag,
#        "eficiencia": eficienciasNoFlag,
#        "speedup": speedupsNoFlag}

# dataFrame = pd.DataFrame(dic)
# dataFrame.to_csv("dados.dat", index=False, sep=tab)


In [ ]:
temposNoFlag = []
eficienciasNoFlag = []
speedupsNoFlag = []

for malha in malhas:
    file = pd.read_csv(f'{malha}/noFlag.out', header = None, sep='\s+')
    
    if ',' in file[1][0]:
        tempo = file[1][0].replace(',', '.')
    else:
        tempo = file[1][0]
    
    tempo = converterTempo(tempo)
    temposNoFlag.append(tempo)
    
for i in range(np.size(cores)):
    eficienciasNoFlag.append((1 - (temposNoFlag[i]/temposNoFlag[0]))*100)
    
for i in range(np.size(cores)):
    speedupsNoFlag.append((temposNoFlag[0]/temposNoFlag[i]))
       
print(temposNoFlag)
print(eficienciasNoFlag)
print(speedupsNoFlag)